# oneM2M - Subscriptions and Notifications

This notebook demonstrates how to subscribe to notifications from resources.

 <div class="alert alert-block alert-info" style="background-color: white; border: 2px solid; padding: 10px; width: calc(100% - 600px);">
    <b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; Note</b><br>
Please execute the notebook "oneM2M - Subscriptions and Notifications - Notification Server" first from <a href="start-notificationServer.ipynb">notification server notebook</a> first.<br>
This starts a notification server that will receive the notifications that result from the following requests.
<br>
If you later see an error like "internal server errror" and a <i>Result Status Code</i> of <i>5204</i> you most likely forget to start the notification server. Please open the <a href="start-notificationServer.ipynb">notification server notebook</a> and run or restart the server.
</div>


- Create a &lt;subscription> resource
- Create &lt;contentInstance> resource and send a notification


## Intitialization
The section does import necessary modules and configurations, and prepares the CSE for this notebook.


In [ ]:
%run src/init.py subscriptions

## Subscribe to a &lt;container> Resource

<img align="right" src="images/06/create_subscription.png">



The following requests creates a &lt;subscription> resource for the *Container* resource.


<div class="alert alert-block alert-warning" style="background-color: white; border: 2px solid; padding: 10px; width: calc(100% - 600px);">
    <b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; oneM2M</b><br>
&lt;subscription> resource are used to monitor resource creations, updates and more. They also carry the configuration when and how the notifications are sent to a notification target.
</div>


These attributes are part of the request:

- **nu** : The Notification URI specifies where to send verification and notification requests. This points to the Notification Server in the other running notebook (s.a.)
- **nct** : The Notification Content Type attribute specifies the kind of content that should be sent in a notification initiated by this subscription. Possible values are:

| Value | Interpretation                           |
|-------|:-----------------------------------------|
| 1     | All. The whole resource                  |
| 2     | Only modified attributes                 |
| 3     | The resource's *resource ID*             |
| 4     | The trigger payload in case of Triggers  |

- **exc** : The Expiration Counter indicates that the subscriber wants to set the life of this subscription to a limit of a maximum number of notifications.
- **enc/net** : The Notification Event Types indicates the reason for which a notification is to be generated. This can be a list of reasons taken from the following list:

| Value | Interpretation                                        |
|-------|:------------------------------------------------------|
| 1     | Update of Resource                                    |
| 2     | Delete of Resource                                    |
| 3     | Create of Direct Child Resource                       |
| 4     | Delete of Direct Child Resource                       |
| 5     | Retrieve of Container Resource With No Child Resource |
| 6     | Trigger Received For AE_Resource                      |
| 7     | Blocking Update                                       |



In [ ]:
CREATE (                                           # CREATE request
    
    # Create subscription resource under the container
    target                  = cseBaseName + '/Notebook-AE/Container',  
    
    # Request Attributes
    originator              = 'Cmyself',           # Set the originator
    requestIdentifier       = '123',               # Unique request identifier
    releaseVersionIndicator = '3',                 # Release version indicator
    resourceType            = Type.Subscription,   # Type of the resource: Subscription
    
    # Request Body
    content = 
        {
            'm2m:sub': {
                'rn'  : 'Subscription',
                'nu'  : [ notificationURL ],
                'nct' : 1,
                'enc' : {
                    'net': [ 1, 2, 3, 4 ]
                }
            }
        }
)

**Please check the [Notification Server](start-notificationServer.ipynb) notebook** (the one with the running Notification Server) to see the verification request.  
When this requests returns with a *201 (Created)* status code / *2001 oneM2M Response Status Code* then the creation of the &lt;subscription> resource as well as the verification request were successfull.


 <div class="alert alert-block alert-info" style="background-color: white; border: 2px solid; padding: 10px; width: calc(100% - 600px);">
    <b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; Note</b><br>
    If you see an error like "Verification request failed" or "internal server errror" and a <i>Response Status Code</i> of 5204 you most likely forget to start the notification server. Please open the <a href="start-notificationServer.ipynb">notification server</a> notebook and run the server.<br>
    <b>You must then execute the above cell again!</b>
</div>




<img align="right" src="images/06/create_contentInstance.png">


## Add a &lt;contentInstance> Resource and Send a Notification

We now have subscribed to notifications whenever the &lt;container> changes. We can raise a notification by adding a new &lt;contentInstance> resource to the container with the following request.

In [ ]:
CREATE (                                             # CREATE request
    
    # Add the <container> under the <AE>
    target                  = cseBaseName + '/Notebook-AE/Container',
    
    # Request Attributes
    originator              = 'Cmyself',             # Set the originator
    requestIdentifier       = '123',                 # Unique request identifier
    releaseVersionIndicator = '3',                   # Release version indicator
    resourceType            = Type.ContentInstance,  # Type of the resource (ContentInstance)
    
    # Request Body
    content = 
        {
            'm2m:cin' : {
                'cnf' : 'text/plain:0',
                'con' : 'Good news, everybody!'
            }
        }
)

**Again, please check the [Notification Server](start-notificationServer.ipynb) notebook to see the notification.** 

<div class="alert alert-block alert-warning" style="background-color: white; border: 2px solid; padding: 10px; width: calc(100% - 600px);">
    <b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; oneM2M</b><br>
This request and result are similar as the previously request when we added a &lt;container> resource. But when we look at the output in the <a href="start-notificationServer.ipynb">Nnotification Server</a> notebook, we see that the notification contains the same resource as this request. This way any receiver of a notification can react to a change of a resource without polling a CSE.<br>
This behaviour can be changed when configuring the &lt;subscription> resource. </div>

Any further creation of a &lt;contentInstance> resource will cause a new notification to be sent.


&nbsp;